# Mutli-Run Evaluation

In [1]:
import sys
import os
# Make sure the root or 03_optimization folder is in the path
sys.path.append(os.path.abspath('../'))   # adjust path if needed
sys.path.append(os.path.abspath('../..')) 

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

from evaluation.evaluator import Evaluator
from evaluation.multi_run_evaluator import MultiRunEvaluator

In [4]:
#time_of_executions = ["2025-08-12_14-13", "2025-08-12_14-22", "2025-08-12_14-55", "2025-08-12_15-17", "2025-08-12_16-12"]

time_of_executions = ["2025-08-14_13-01", "2025-08-14_15-34"] # 3 days all models (ideal weird) => gt_freq=15min
#time_of_executions = ["2025-08-14_15-59", "2025-08-14_16-04"] # 2 days all models (gt_freq=60min)
time_of_executions = ["2025-08-14_17-20"] # 3 days all models gt_freq=60min
csvs = [p for t in time_of_executions for p in Path("../runs").glob(f"logs_{t}*.csv")]

mre = MultiRunEvaluator(run_paths=csvs)
mre.leaderboard()


self.run_paths: [PosixPath('../runs/logs_2025-08-14_17-20_op-interval_freq-60_building-SFH3_run-5798acbe.csv'), PosixPath('../runs/logs_2025-08-14_17-20_op-ideal_freq-60_building-SFH3_run-d4093f0d.csv'), PosixPath('../runs/logs_2025-08-14_17-20_op-mpc_det_freq-60_building-SFH3_run-e2281aa0.csv'), PosixPath('../runs/logs_2025-08-14_17-20_op-mpc_prob_freq-60_building-SFH3_run-5e6c61db.csv'), PosixPath('../runs/logs_2025-08-14_17-20_op-rule-based_freq-60_building-SFH3_run-bf0eb060.csv')]


,model,building,freq,t_start,t_end,pg_import_total,pg_export_total,e_end,net_cost
0,ideal,SFH3_run-d4093f0d,60,2020-05-13 07:00:00+00:00,2020-05-16 06:00:00+00:00,0.000000,-77.877803,0.0000,-6.022305
1,interval,SFH3_run-5798acbe,60,2020-05-13 07:00:00+00:00,2020-05-16 06:00:00+00:00,2.143090,-79.686944,0.6112,-5.562245
2,mpc_prob,SFH3_run-5e6c61db,60,2020-05-13 07:00:00+00:00,2020-05-16 06:00:00+00:00,2.345371,-80.633451,0.0000,-5.535306
3,rule-based,SFH3_run-bf0eb060,60,2020-05-13 07:00:00+00:00,2020-05-16 06:00:00+00:00,0.000000,-75.026079,4.0877,-5.301696
4,mpc_det,SFH3_run-e2281aa0,60,2020-05-13 07:00:00+00:00,2020-05-16 06:00:00+00:00,9.091727,-87.252040,0.0000,-4.591516


In [3]:
mre.pivot()

model,interval,mpc_det,mpc_prob,rule-based
freq,60,60,60,60
building,,,,
SFH3_run-2b3d02d6,NaN,NaN,NaN,-3.006005
SFH3_run-2f4d27a4,-4.268382,NaN,NaN,NaN
SFH3_run-d416c13d,NaN,-3.018229,NaN,NaN
SFH3_run-febef116,NaN,NaN,-4.020126,NaN


## Test the evaluation class

In [ ]:
run_path = Path('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv')
run_path_rb = Path('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')

prices_path = Path('../../01_data/price_data/electricity_price_estimation.csv')
prices_path_const = Path('../../01_data/price_data/const_prices_testing.csv')
ev1 = Evaluator(run_path, prices_path)
ev1.get_costs()

{'import_cost': 0.0,
 'export_revenue': 2.344861855120558,
 'net_cost': -2.344861855120558}

In [7]:
ev2 = Evaluator(run_path_rb, prices_path)
ev2.get_costs()

{'import_cost': 0.0,
 'export_revenue': 2.186811788658414,
 'net_cost': -2.186811788658414}

In [8]:
ev1.get_df()

,action,pb_low,pb_high,pg,gt,soe_now,soe_new,import_price,export_price,costs_buy,costs_sell,cashflow
timestamp,,,,,,,,,,,,
2020-05-13 07:00:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.511872,-2.834009,4.000000,4.076507,0.1139,0.0465,0.0,0.116802,-0.116802
2020-05-13 07:15:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.557424,-2.879560,4.076507,4.153015,0.1139,0.0465,0.0,0.118920,-0.118920
2020-05-13 07:30:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.854330,-3.176466,4.153015,4.229522,0.1139,0.0465,0.0,0.132726,-0.132726
2020-05-13 07:45:00+00:00,-0.322137,-0.322137,-1.201852e-07,-3.387559,-3.709695,4.229522,4.306030,0.1139,0.0465,0.0,0.157521,-0.157521
2020-05-13 08:00:00+00:00,0.655249,0.655249,1.088511e+00,-4.045914,-3.390665,4.306030,4.133596,0.1138,0.0465,0.0,0.188135,-0.188135
2020-05-13 08:15:00+00:00,0.655249,0.655249,1.088511e+00,-4.228103,-3.572853,4.133596,3.961162,0.1138,0.0465,0.0,0.196607,-0.196607
2020-05-13 08:30:00+00:00,0.655249,0.655249,1.088511e+00,-5.395369,-4.740119,3.961162,3.788728,0.1138,0.0465,0.0,0.250885,-0.250885
2020-05-13 08:45:00+00:00,0.655249,0.655249,1.088511e+00,-5.698039,-5.042790,3.788728,3.616294,0.1138,0.0465,0.0,0.264959,-0.264959
2020-05-13 09:00:00+00:00,-0.556352,-0.556352,-1.791756e-06,-4.561881,-5.118234,3.616294,3.748427,0.1094,0.0448,0.0,0.204372,-0.204372


In [9]:
ev1.plot_battery_soe(max_soe=8.0)

In [10]:
ev2.plot_battery_soe(max_soe=8.0)

## Test Multi-Run Evaluator

In [11]:
path1 = Path('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv')
path2 = Path('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')

path_list = [path1, path2]

mre = MultiRunEvaluator(path_list)

self.run_paths: [WindowsPath('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv'), WindowsPath('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')]


In [12]:
mre.leaderboard(by="net_cost")

,model,building,freq,t_start,t_end,net_cost
0,interval,SFH3,60,2020-05-13 07:00:00+00:00,2020-05-13 09:45:00+00:00,-2.344862
1,rule-based,SFH3,60,2020-05-13 07:00:00+00:00,2020-05-13 09:45:00+00:00,-2.186812


In [13]:
mre.pivot()

model,interval,rule-based
freq,60,60
building,,
SFH3,-2.344862,-2.186812
